In [1]:
import re
import pandas as pd
from collections import OrderedDict

In [2]:
docs_df = pd.read_csv('../data/law_analyzer_new4.csv')
docs_df

,_index,_id,_score,_source.Status,_source.No,_source.PeraturanGoId,_source.Blocks,_source.TanggalPenetapan,_source.Bidang,_source.Slug,...,_source.Tahun,_source.TanggalPengundangan,_source.Bentuk,_source.PeraturanId,_source.BlocksMinioPath,_source.FileMinioPath,_source.Domain,_source.Tematik,_source.TanggalPembaruan,_source.BlocksMiniPath
0,law_analyzer_new4,221-pmk.010-2015,1.0,Berlaku,221.0,https://peraturan.go.id/files/bn1843-2015.pdf,"[{'ref': 'none', 'bab': 'none', 'pasal': 'pasa...",2015-12-07 00:00:00.000,NaN,221/pmk.010/2015,...,2015.0,2015-12-07 00:00:00.000,Peraturan Menteri Keuangan,CCDD282D-D3A1-43A4-BE32-94C635A50127,NaN,NaN,NaN,NaN,NaN,NaN
1,law_analyzer_new4,pp-21-tahun-2005,1.0,Berlaku,21.0,https://peraturan.go.id/files/pp19-2005.pdf,"[{'ref': 'none', 'bab': 'bab-i', 'pasal': 'pas...",2005-05-19 00:00:00.000,NaN,pp-21-tahun-2005,...,2005.0,2005-05-19 00:00:00.000,Peraturan Pemerintah,3990F6C9-792A-43A1-93A3-736B9A11779E,NaN,NaN,NaN,NaN,NaN,NaN
2,law_analyzer_new4,pp-51-tahun-2008,1.0,Berlaku,51.0,NaN,"[{'ref': 'none', 'bab': 'none', 'pasal': 'pasa...",2008-01-01 00:00:00.000,NaN,pp-51-tahun-2008,...,2008.0,2008-01-01 00:00:00.000,Peraturan Pemerintah,6086A055-44C0-4C6E-B7D7-D7617EFF4C16,NaN,NaN,NaN,NaN,NaN,NaN
3,law_analyzer_new4,62-pmk.04-2018,1.0,Tidak Berlaku,62.0,https://peraturan.go.id/files/bn777-2018.pdf,"[{'ref': 'none', 'bab': 'none', 'pasal': 'pasa...",2018-06-06 00:00:00.000,NaN,62/pmk.04/2018,...,2018.0,2018-06-21 00:21:00.000,Peraturan Menteri Keuangan,BE35AA04-E741-4264-8914-4AB922C47318,NaN,NaN,NaN,NaN,NaN,NaN
4,law_analyzer_new4,42-pmk.05-2017,1.0,Berlaku,42.0,https://peraturan.go.id/files/bn400-2017.pdf,"[{'ref': 'none', 'bab': 'none', 'pasal': 'pasa...",2017-03-10 00:00:00.000,NaN,42/pmk.05/2017,...,2017.0,2017-03-10 00:00:00.000,Peraturan Menteri Keuangan,CD4FDA39-BF00-4C8E-AD91-5A103139976B,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3020,law_analyzer_new4,pp-9-tahun-2014,1.0,Berlaku,9.0,https://peraturan.go.id/files/pp9-2014bt.pdf,"[{'ref': 'none', 'bab': 'bab-i', 'pasal': 'pas...",2014-02-12 00:00:00.000,NaN,pp-9-tahun-2014,...,2014.0,2014-02-12 00:00:00.000,Peraturan Pemerintah,BAEBDE95-A779-4D45-9056-247C47CBC502,la_parse/BAEBDE95-A779-4D45-9056-247C47CBC502....,NaN,NaN,NaN,NaN,NaN
3021,law_analyzer_new4,pp-25-tahun-2017,1.0,Berlaku,25.0,https://peraturan.go.id/files/pp25-2017bt.pdf,"[{'ref': 'none', 'bab': 'none', 'pasal': 'pasa...",2017-06-13 00:00:00.000,NaN,pp-25-tahun-2017,...,2017.0,2017-06-13 00:00:00.000,Peraturan Pemerintah,7BEBBDD3-FFE0-4E45-9458-4B9A641B2839,la_parse/7BEBBDD3-FFE0-4E45-9458-4B9A641B2839....,NaN,NaN,NaN,NaN,NaN
3022,law_analyzer_new4,160-pmk.04-2010,1.0,Tidak Berlaku,160.0,NaN,"[{'ref': 'none', 'bab': 'none', 'pasal': 'pasa...",2010-09-01 00:00:00.000,Hukum Keuangan Negara,160/pmk.04/2010,...,2010.0,2010-09-01 00:00:00.000,Peraturan Menteri Keuangan,A78C2FD3-F971-4ECA-A03C-71ED1225E466,la_parse/A78C2FD3-F971-4ECA-A03C-71ED1225E466....,NaN,NaN,NaN,NaN,NaN
3023,law_analyzer_new4,146-pmk.05-2019,1.0,Tidak Berlaku,146.0,https://peraturan.go.id/files/BN+1224-2019.pdf,"[{'content': ' '}, {'content': 'BERITA NEGARA ...",2019-10-18 00:00:00.000,NaN,146/pmk.05/2019,...,2019.0,NaN,Peraturan Menteri Keuangan,E445A109-5781-4C02-B203-DDF63F684C6B,parse_la/bb77bd83-675f-4537-9c32-9d83aead220c....,document/E445A109-5781-4C02-B203-DDF63F684C6B.pdf,NaN,NaN,NaN,NaN


In [3]:
invalid_docs_id = []

for index, row in docs_df.iterrows():
    if type(row['_source.Blocks']) != str:
        invalid_docs_id.append(row['_id'])

invalid_docs_id

['coba2-upload-pdf', 'undefined', 'conothbaru']

In [4]:
# drop the invalid documents
docs_df = docs_df[~docs_df['_id'].isin(invalid_docs_id)]

# convert all _source.Blocks to list
docs_df['_source.Blocks'] = docs_df['_source.Blocks'].apply(eval)

C:\Users\zoxrl\AppData\Local\Temp\ipykernel_14408\717057130.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  docs_df['_source.Blocks'] = docs_df['_source.Blocks'].apply(eval)


In [5]:
# create new dataframe with only the relevant fields (_id, _source.Judul, _source.Blocks)
data_df = docs_df[['_id','_source.Blocks', '_source.Nomor', '_source.Judul', '_source.Tahun', '_source.No']]
data_df

,_id,_source.Blocks,_source.Nomor,_source.Judul,_source.Tahun,_source.No
0,221-pmk.010-2015,"[{'ref': 'none', 'bab': 'none', 'pasal': 'pasa...",221/PMK.010/2015,Pengenaan Bea Masuk Anti Dumping Terhadap Impo...,2015.0,221.0
1,pp-21-tahun-2005,"[{'ref': 'none', 'bab': 'bab-i', 'pasal': 'pas...",PP 21 TAHUN 2005,Ahli Teknologi Kekayaan Intelektual Serta Hasi...,2005.0,21.0
2,pp-51-tahun-2008,"[{'ref': 'none', 'bab': 'none', 'pasal': 'pasa...",PP 51 TAHUN 2008,Pajak Penghasilan atas Penghasilan Dari Usaha ...,2008.0,51.0
3,62-pmk.04-2018,"[{'ref': 'none', 'bab': 'none', 'pasal': 'pasa...",62/PMK.04/2018,Perubahan Kedua atas Peraturan Menteri Keuanga...,2018.0,62.0
4,42-pmk.05-2017,"[{'ref': 'none', 'bab': 'none', 'pasal': 'pasa...",42/PMK.05/2017,Perubahan atas Peraturan Menteri Keuangan Nomo...,2017.0,42.0
...,...,...,...,...,...,...
3020,pp-9-tahun-2014,"[{'ref': 'none', 'bab': 'bab-i', 'pasal': 'pas...",PP 9 TAHUN 2014,Pelaksanaan Undang-Undang Nomor 4 Tahun 2011 t...,2014.0,9.0
3021,pp-25-tahun-2017,"[{'ref': 'none', 'bab': 'none', 'pasal': 'pasa...",PP 25 TAHUN 2017,Pemberian Tunjangan Hari Raya dalam Tahun Angg...,2017.0,25.0
3022,160-pmk.04-2010,"[{'ref': 'none', 'bab': 'none', 'pasal': 'pasa...",160/PMK.04/2010,Nilai Pabean untuk Penghitungan Bea Masuk.,2010.0,160.0
3023,146-pmk.05-2019,"[{'content': ' '}, {'content': 'BERITA NEGARA ...",146/PMK.05/2019,Tarif Layanan Badan Layanan Umum Politeknik Ke...,2019.0,146.0


In [6]:
def DetectDefinition(Text):
    Keywords = ["adalah", "merupakan", "yang selanjutnya disebut", "yang selanjutnya disingkat"]
    return any(Keyword in Text for Keyword in Keywords)

def ExtractDefinition(Text):
    Parts = Text.split("adalah")
    if len(Parts) == 1:
        Parts = Text.split("merupakan")
    if len(Parts) == 1:
        Parts = Text.split("yang selanjutnya disebut")
    if len(Parts) == 1:
        Parts = Text.split("yang selanjutnya disingkat")
    return Parts[-1].strip()

def DetectDefinitionsInText(Text):
    Sentences = re.split(r'\b\d+\.\s*', Text)
    Sentences = [Sentence.strip()
                for Sentence in Sentences if Sentence.strip()]

    Results = []
    for Sentence in Sentences:
        Sentence = re.sub(r'\s+', ' ', Sentence)

        if DetectDefinition(Sentence):
            Term = None
            if "yang selanjutnya disebut" in Sentence:
                Term = Sentence.split("yang selanjutnya disebut")[0].strip()
            elif "yang selanjutnya disingkat" in Sentence:
                Term = Sentence.split("yang selanjutnya disingkat")[0].strip()
            elif "adalah" in Sentence:
                Term = Sentence.split("adalah")[0].strip()
            elif "merupakan" in Sentence:
                Term = Sentence.split("merupakan")[0].strip()

            Definition = ExtractDefinition(Sentence)

            ShortTerm = None
            if "yang selanjutnya disebut" in Sentence:
                ShortTerm = re.search(
                    r'yang selanjutnya disebut(.*?)adalah', Sentence)
            elif "yang selanjutnya disingkat" in Sentence:
                ShortTerm = re.search(
                    r'yang selanjutnya disingkat(.*?)merupakan', Sentence)

            if ShortTerm:
                ShortTerm = ShortTerm.group(1).strip()

            result = {
                "Text": Sentence,
                "Term": Term,
                "ShortTerm": ShortTerm,
                "Definition": Definition,
                "Type": "DEFINITION"
            }
            Results.append(result)

    return Results

In [7]:
def title_prep(context):
    context_parts = context.split(" - ")
    return context_parts[0].strip()


def context_prep(context):
    context_parts = [part.strip() for part in context.split(" - ")]
    return context_parts


def heading_prep(heading_list):
    list_1d = [item for sublist in heading_list for item in sublist]
    unique_heading_list = list(OrderedDict((x, None) for x in list_1d))

    if len(unique_heading_list) > 1:
        unique_heading_list = unique_heading_list[1:]
    else:
        unique_heading_list = None

    return unique_heading_list

In [8]:
docs_title_list = []

valid_docs_content_id = []
invalid_docs_content_id = []

for index, row in data_df.iterrows():
    for c in row['_source.Blocks']:
        if 'type' in c and c['type'] == 'CONTENT_PASAL':
            title_text = title_prep(c['context'])
            docs_title_list.append(title_text)
            valid_docs_content_id.append(row['_id'])
            break
        else:
            if row['_id'] not in invalid_docs_content_id:
                invalid_docs_content_id.append(row['_id'])

# remove id in invalid_docs_title_id if exists in valid_docs_title_id
invalid_docs_content_id = [x for x in invalid_docs_content_id if x not in valid_docs_content_id]
invalid_docs_content_id

['uu-10-tahun-1994', 'uu-6-tahun-1966', 'uu-16-tahun-2000']

In [9]:
# delete the invalid documents
data_df = data_df[~data_df['_id'].isin(invalid_docs_content_id)]

# reset index
data_df = data_df.reset_index(drop=True)

In [10]:
docs_definisi_list = []

for index, row in data_df.iterrows():
    doc_heading_list = []
    context_temp = []

    for c in row['_source.Blocks']:
        if 'pasal' in c and c['pasal'] == 'pasal-1':
            definition_term = DetectDefinitionsInText(c['content'])
            if definition_term:
                term_list = [term['Term'] for term in definition_term]
                docs_definisi_list.append(term_list)
            else:
                docs_definisi_list.append(None)
            break
        else:
            docs_definisi_list.append(None)
            break

docs_definisi_list

[None,
 ['Standar nasional pendidikan',
  'Pendidikan formal',
  'Pendidikan nonformal',
  'Standar kompetensi lulusan',
  'Standar isi',
  'Standar proses',
  'Standar pendidik dan tenaga kependidikan',
  'Standar sarana dan prasarana',
  'Standar pengelolaan',
  'Standar pembiayaan',
  'Standar penilaian pendidikan',
  'Biaya operasi satuan pendidikan',
  'Kurikulum',
  'Kerangka dasar kurikulum',
  'Kurikulum tingkat satuan pendidikan',
  'Peserta didik',
  'Penilaian',
  'Evaluasi pendidikan',
  'Ulangan',
  'Ujian',
  'Akreditasi'],
 None,
 None,
 None,
 ['Bagian Anggaran Bendahara Umum Negara',
  'Likuidasi',
  'Entitas Akuntansi',
  'Entitas Akuntansi yang Ditunjuk',
  'Identitas Entitas Akuntansi',
  'Neraca',
  'Laporan Realisasi Anggaran',
  'Laporan Operasional',
  'Laporan Perubahan Ekuitas',
  'Catatan atas Laporan Keuangan',
  'Kantor Pelayanan Perbendaharaan Negara'],
 ['Pajak Rokok',
  'Rokok',
  'Cukai Rokok',
  'Surat Pemberitahuan Pajak Rokok',
  'Permohonan Pemesana

In [11]:
docs_heading_list = []
docs_id_list = []

for index, row in data_df.iterrows():
    doc_heading_list = []
    context_temp = []

    for c in row['_source.Blocks']:
        if 'type' in c and c['type'] == 'CONTENT_PASAL':
            context_text = context_prep(c['context'])
            
            if context_text not in context_temp:
                doc_heading_list.append(context_text)
                context_temp.append(context_text)

    docs_heading_list.append(doc_heading_list)
    docs_id_list.append(row['_id'])

In [12]:
# apply heading_prep to all documents
docs_heading_list = [heading_prep(heading_list) for heading_list in docs_heading_list]
docs_heading_list

[None,
 ['KETENTUAN UMUM',
  'LINGKUP, FUNGSI, DAN TUJUAN',
  'STANDAR ISI',
  'Umum',
  'Kerangka Dasar dan Struktur Kurikulum',
  'Kurikulum Tingkat Satuan Pendidikan',
  'Kalender Pendidikan/Akademik',
  'STANDAR PROSES',
  'STANDAR KOMPETENSI LULUSAN',
  'STANDAR PENDIDIK DAN TENAGA KEPENDIDIKAN',
  'Penilaian Hasil Belajar oleh Pendidik',
  'KETENTUAN PERALIHAN',
  'KETENTUAN PENUTUP'],
 None,
 None,
 None,
 ['KETENTUAN UMUM',
  'RUANG LINGKUP',
  'KRITERIA LIKUIDASI',
  'LIKUIDASI ENTITAS AKUNTANSI',
  'Penanggung Jawab Likuidasi',
  'Penetapan Entitas Akuntansi Yang Ditunjuk',
  'Tahapan Likuidasi',
  'Penyelesaian Hak dan Kewajiban',
  'Hak dan Kewajiban',
  'Penyelesaian Saldo Kas dan Gaji Bulan Berikutnya',
  'Penyelesaian Piutang dan Utang',
  'Penyelesaian Investasi',
  'Penyelesaian Persediaan, Aset Tetap, dan Aset Lainnya',
  'Penyusunan Laporan Keuangan',
  'PERNYATAAN TANGGUNG JAWAB',
  'KETENTUAN PENUTUP'],
 ['KETENTUAN UMUM',
  'TATA CARA PEMUNGUTAN PAJAK ROKOK',
  'P

In [13]:
# create dictionary to map the id to the content
id_to_definisi = dict(zip(docs_id_list, docs_definisi_list))
id_to_heading = dict(zip(docs_id_list, docs_heading_list))

# add the content to the dataframe
data_df['definisi'] = data_df['_id'].map(id_to_definisi)
data_df['heading'] = data_df['_id'].map(id_to_heading)

# delete the _source.Blocks & _id column
data_df = data_df.drop(columns=['_id', '_source.Blocks'])
data_df

,_source.Nomor,_source.Judul,_source.Tahun,_source.No,definisi,heading
0,221/PMK.010/2015,Pengenaan Bea Masuk Anti Dumping Terhadap Impo...,2015.0,221.0,None,None
1,PP 21 TAHUN 2005,Ahli Teknologi Kekayaan Intelektual Serta Hasi...,2005.0,21.0,"[Standar nasional pendidikan, Pendidikan forma...","[KETENTUAN UMUM, LINGKUP, FUNGSI, DAN TUJUAN, ..."
2,PP 51 TAHUN 2008,Pajak Penghasilan atas Penghasilan Dari Usaha ...,2008.0,51.0,None,None
3,62/PMK.04/2018,Perubahan Kedua atas Peraturan Menteri Keuanga...,2018.0,62.0,None,None
4,42/PMK.05/2017,Perubahan atas Peraturan Menteri Keuangan Nomo...,2017.0,42.0,None,None
...,...,...,...,...,...,...
3014,PP 9 TAHUN 2014,Pelaksanaan Undang-Undang Nomor 4 Tahun 2011 t...,2014.0,9.0,"[Geospasial, Data Geospasial, Informasi Geospa...","[KETENTUAN UMUM, PENYELENGGARAAN INFORMASI GEO..."
3015,PP 25 TAHUN 2017,Pemberian Tunjangan Hari Raya dalam Tahun Angg...,2017.0,25.0,"[Pegawai Negeri Sipil, Prajurit Tentara Nasion...",None
3016,160/PMK.04/2010,Nilai Pabean untuk Penghitungan Bea Masuk.,2010.0,160.0,None,None
3017,146/PMK.05/2019,Tarif Layanan Badan Layanan Umum Politeknik Ke...,2019.0,146.0,None,None


In [14]:
# rename the columns
data_df = data_df.rename(columns={'_source.Nomor': 'nomor_peraturan', '_source.Judul': 'judul', '_source.Tahun': 'tahun', '_source.No': 'no'})

# convert float to int for tahun and no column
data_df['tahun'] = data_df['tahun'].astype(int)
data_df['no'] = data_df['no'].astype(int)

data_df

,nomor_peraturan,judul,tahun,no,definisi,heading
0,221/PMK.010/2015,Pengenaan Bea Masuk Anti Dumping Terhadap Impo...,2015,221,None,None
1,PP 21 TAHUN 2005,Ahli Teknologi Kekayaan Intelektual Serta Hasi...,2005,21,"[Standar nasional pendidikan, Pendidikan forma...","[KETENTUAN UMUM, LINGKUP, FUNGSI, DAN TUJUAN, ..."
2,PP 51 TAHUN 2008,Pajak Penghasilan atas Penghasilan Dari Usaha ...,2008,51,None,None
3,62/PMK.04/2018,Perubahan Kedua atas Peraturan Menteri Keuanga...,2018,62,None,None
4,42/PMK.05/2017,Perubahan atas Peraturan Menteri Keuangan Nomo...,2017,42,None,None
...,...,...,...,...,...,...
3014,PP 9 TAHUN 2014,Pelaksanaan Undang-Undang Nomor 4 Tahun 2011 t...,2014,9,"[Geospasial, Data Geospasial, Informasi Geospa...","[KETENTUAN UMUM, PENYELENGGARAAN INFORMASI GEO..."
3015,PP 25 TAHUN 2017,Pemberian Tunjangan Hari Raya dalam Tahun Angg...,2017,25,"[Pegawai Negeri Sipil, Prajurit Tentara Nasion...",None
3016,160/PMK.04/2010,Nilai Pabean untuk Penghitungan Bea Masuk.,2010,160,None,None
3017,146/PMK.05/2019,Tarif Layanan Badan Layanan Umum Politeknik Ke...,2019,146,None,None


In [15]:
# save the dataframe to csv
data_df.to_csv('../data/law_analyzer_new4_keywords.csv', index=False)